In [17]:
# prompt: help me unzip the contents of archive.zip

import zipfile

with zipfile.ZipFile('/content/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')


In [18]:
# Mutual Funds Risk Indicator - Jupyter Notebook Version

# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Step 2: Load the Dataset (downloaded from Kaggle)
df = pd.read_csv("Mutual_Funds.csv")

# Step 3: Clean and Preprocess Data
df.rename(columns={
    'Scheme Code': 'Scheme_Code',
    'Scheme Name': 'Scheme_Name',
    'Net Asset Value': 'NAV',
    'Date': 'Date'
}, inplace=True)

df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df.sort_values(by=['Scheme_Code', 'Date'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Step 4: Feature Engineering
df['Daily_Return'] = df.groupby('Scheme_Code')['NAV'].pct_change()
df['Rolling_Std_30'] = df.groupby('Scheme_Code')['Daily_Return'].rolling(window=30).std().reset_index(level=0, drop=True)
df['Mean_Daily_Return'] = df.groupby('Scheme_Code')['Daily_Return'].transform('mean')
df['Std_Daily_Return'] = df.groupby('Scheme_Code')['Daily_Return'].transform('std')
df['Sharpe_Ratio'] = df['Mean_Daily_Return'] / df['Std_Daily_Return']

# Drop rows with missing values
df.dropna(subset=['Mean_Daily_Return', 'Std_Daily_Return', 'Sharpe_Ratio'], inplace=True)

# Step 5: Define Risk Labels
def classify_risk(std):
    if std < 0.01:
        return 'Low'
    elif std < 0.02:
        return 'Medium'
    else:
        return 'High'

df['Risk_Level'] = df['Std_Daily_Return'].apply(classify_risk)

# Step 6: Train Machine Learning Model
features = ['Mean_Daily_Return', 'Std_Daily_Return', 'Sharpe_Ratio']
X = df[features]
y = df['Risk_Level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

# Step 7: Save the Model
import joblib
joblib.dump(model, 'mutual_fund_risk_model.pkl')

# Step 8: Streamlit App Code
streamlit_code = '''
import streamlit as st
import numpy as np
import joblib

model = joblib.load('mutual_fund_risk_model.pkl')

st.title("Mutual Fund Risk Predictor")

mean_return = st.number_input("Mean Daily Return", value=0.0, format="%.5f")
std_return = st.number_input("Standard Deviation of Daily Return", value=0.0, format="%.5f")
sharpe_ratio = st.number_input("Sharpe Ratio", value=0.0, format="%.5f")

if st.button("Predict Risk Level"):
    input_data = np.array([[mean_return, std_return, sharpe_ratio]])
    prediction = model.predict(input_data)
    st.success(f"Predicted Risk Level: {prediction[0]}")
'''

with open("streamlit_app.py", "w") as f:
    f.write(streamlit_code)

print("Streamlit app code saved as streamlit_app.py")


Classification Report:

              precision    recall  f1-score   support

        High       1.00      1.00      1.00    470641
         Low       1.00      1.00      1.00   3856311
      Medium       1.00      1.00      1.00   1066034

    accuracy                           1.00   5392986
   macro avg       1.00      1.00      1.00   5392986
weighted avg       1.00      1.00      1.00   5392986

Confusion Matrix:

[[ 470641       0       0]
 [      0 3856311       0]
 [      0       0 1066034]]
Streamlit app code saved as streamlit_app.py


In [20]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.9 MB/s eta 0:00:00


In [21]:
! wget -q -O - ipv4.icanhazip.com

34.169.225.87


In [25]:
! streamlit run streamlit_app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.225.87:8501

⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://warm-moments-clap.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.p